In [1]:
import optuna
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import naive_bayes
from sklearn import ensemble, tree
import xgboost as xgb
import catboost
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from functools import partial
from skopt import space, gp_minimize
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
import warnings
warnings.filterwarnings('ignore')

# Load dataset and preproccessing.

In [2]:
def remove_high_correlation(dataset, threshold):
    col_names = []
    for i in range(dataset.corr().shape[1] - 1):
        for j in range(i):
            if abs(dataset.corr().iloc[i,j]) >= threshold:
                col_names.append(dataset.corr().columns[i])
    return col_names

df = pd.read_csv('/home/hamza/Downloads/DataSets/All_Classification_Models/WA_Fn-UseC_-HR-Employee-Attrition.csv')

for i in df.select_dtypes(include = [object, bool]).columns:
    df[i] = LabelEncoder().fit_transform(df[i])

for i in df.columns:
    if len(df[i].value_counts()) == 1:
        df.drop(i, axis = 1, inplace = True)

df.drop(remove_high_correlation(df, 0.7), axis = 1, inplace = True)

X = df.drop('Attrition', axis = 1).values
y = df['Attrition'].values


# Support Vector Machine with different hyper parameter tuning techniques.

In [3]:
# apply (Support Vector Machine) with GridSearch.

In [4]:
algo = SVC()

model = model_selection.GridSearchCV(algo, param_grid = {
    'degree' : [i for i in range(1,5)],
    'kernel' : ['rbf','poly','sigmoid'],
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'degree': 1, 'kernel': 'rbf'}
0.8367346938775511


In [5]:
# apply (Support Vector Machine) with RandomSearch.

In [6]:
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'degree' : np.arange(1,5),
    'kernel' : ['poly','rbf','sigmoid']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'kernel': 'poly', 'degree': 1}
0.8367346938775511


In [7]:
# apply (Support Vector Machine) with gp minimize.

In [8]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = SVC(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Integer(1,5, name = 'degree'),
    space.Categorical(['poly','rbf','sigmoid'], name = 'kernel')
]

param_names = ['degree','kernel']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.3022
Function value obtained: -0.8388
Current minimum: -0.8388
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.2049
Function value obtained: -0.8122
Current minimum: -0.8388
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.1990
Function value obtained: -0.8388
Current minimum: -0.8388
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.1955
Function value obtained: -0.8388
Current minimum: -0.8388
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.1956
Function value obtained: -0.8388
Current minimum: -0.8388
Iteration No: 6 started. 

In [9]:
# apply (Support Vector Machine) with hyperopt.

In [10]:
def optimize(params, x, y):
    model = SVC(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'degree': scope.int(hp.quniform('degree',1,5,1)),
    'kernel': hp.choice('kernel',['poly','rbf','sigmoid'])
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:08<00:00,  1.83trial/s, best loss: -0.8387755102040817]
{'degree': 4.0, 'kernel': 0}


In [11]:
# apply (Support Vector Machine) with optuna.

In [12]:
def optimize(trial, x, y):
    kernel = trial.suggest_categorical('kernel', ['poly','rbf','sigmoid'])
    degree = trial.suggest_int('degree', 1, 5)
    model = SVC(
        kernel = kernel,
        degree = degree
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-13 09:20:16,775] A new study created in memory with name: no-name-1e155638-473c-4e00-aef5-7050a53320ce
[I 2022-08-13 09:20:16,996] Trial 0 finished with value: 0.8387755102040817 and parameters: {'kernel': 'rbf', 'degree': 1}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:20:17,191] Trial 1 finished with value: 0.8387755102040817 and parameters: {'kernel': 'rbf', 'degree': 2}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:20:18,155] Trial 2 finished with value: 0.8387755102040817 and parameters: {'kernel': 'poly', 'degree': 4}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:20:18,357] Trial 3 finished with value: 0.8122448979591838 and parameters: {'kernel': 'sigmoid', 'degree': 2}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:20:18,560] Trial 4 finished with value: 0.8122448979591838 and parameters: {'kernel': 'sigmoid', 'degree': 3}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:

{'kernel': 'rbf', 'degree': 1} 0.8387755102040817


# K Nearest Neighbor with different hyper parameter tuning techniques.

In [13]:
# apply K Nearest Neighbor with GridSearch.

In [14]:
algo = KNeighborsClassifier()

model = model_selection.GridSearchCV(algo, param_grid = {
    'n_neighbors' : [i for i in range(5,101)],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric' : ['euclidean', 'manhattan']
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 11}
0.8374149659863945


In [15]:
# apply K Nearest Neighbor with RandomeSearch.

In [16]:
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'n_neighbors' : np.arange(5,100),
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric' : ['euclidean', 'manhattan']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'n_neighbors': 87, 'metric': 'euclidean', 'algorithm': 'brute'}
0.8367346938775511


In [17]:
# apply K Nearest Neighbor with gp minimize.

In [18]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = KNeighborsClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Integer(5,100, name = 'n_neighbors'),
    space.Categorical(['auto', 'ball_tree', 'kd_tree', 'brute'], name = 'algorithm')
    
]

param_names = ['n_neighbors','algorithm']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.1250
Function value obtained: -0.8252
Current minimum: -0.8252
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.1864
Function value obtained: -0.8388
Current minimum: -0.8388
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.1847
Function value obtained: -0.8388
Current minimum: -0.8388
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.1267
Function value obtained: -0.8388
Current minimum: -0.8388
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.1846
Function value obtained: -0.8388
Current minimum: -0.8388
Iteration No: 6 started. 

In [19]:
# apply K Nearest Neighbor with hyperopt.

In [20]:
def optimize(params, x, y):
    model = KNeighborsClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'n_neighbors': scope.int(hp.quniform('n_neighbors',5,100,1)),
    'algorithm': hp.choice('algorithm',['auto', 'ball_tree', 'kd_tree', 'brute']),
    'metric': hp.choice('metric',['euclidean', 'manhattan'])
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:01<00:00, 11.36trial/s, best loss: -0.8387755102040817]
{'algorithm': 0, 'metric': 1, 'n_neighbors': 50.0}


In [21]:
# apply K Nearest Neighbor with optuna.

In [22]:
def optimize(trial, x, y):
    n_neighbors = trial.suggest_int('n_neighbors', 5,100)
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan'])
    model = KNeighborsClassifier(
        n_neighbors = n_neighbors,
        algorithm = algorithm,
        metric = metric
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-13 09:21:52,640] A new study created in memory with name: no-name-c3aa3702-267b-49da-af4d-20c6d1f7844a
[I 2022-08-13 09:21:52,751] Trial 0 finished with value: 0.8387755102040817 and parameters: {'n_neighbors': 88, 'algorithm': 'ball_tree', 'metric': 'euclidean'}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:21:52,822] Trial 1 finished with value: 0.8387755102040817 and parameters: {'n_neighbors': 94, 'algorithm': 'ball_tree', 'metric': 'euclidean'}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:21:52,887] Trial 2 finished with value: 0.8380952380952381 and parameters: {'n_neighbors': 16, 'algorithm': 'kd_tree', 'metric': 'manhattan'}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:21:52,945] Trial 3 finished with value: 0.8387755102040817 and parameters: {'n_neighbors': 16, 'algorithm': 'ball_tree', 'metric': 'euclidean'}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:21:53,012] Trial 4 finished wit

{'n_neighbors': 88, 'algorithm': 'ball_tree', 'metric': 'euclidean'} 0.8387755102040817


# Naive Bayes with different hyper parameter tuning techniques.

In [23]:
# apply Naive Bayes with GridSearch.

In [24]:
algo = naive_bayes.BernoulliNB()

model = model_selection.GridSearchCV(algo, param_grid = {
    'alpha' : [i for i in range(1,11)],
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('BernoulliNB: ')
print(model.best_params_)
print(model.best_score_)
#########################
algo = naive_bayes.MultinomialNB()

model = model_selection.GridSearchCV(algo, param_grid = {
    'alpha' : [i for i in range(1,11)],
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('MultinomialNB: ')
print(model.best_params_)
print(model.best_score_)
##########################
algo = naive_bayes.GaussianNB()

model = model_selection.GridSearchCV(algo, param_grid = {
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('GaussianNB: ')
print(model.best_params_)
print(model.best_score_)

BernoulliNB: 
{'alpha': 7}
0.8523809523809524
MultinomialNB: 
{'alpha': 1}
0.535374149659864
GaussianNB: 
{}
0.8619047619047618


In [25]:
# apply Naive Bayes with RandomSearch.

In [26]:
algo = naive_bayes.BernoulliNB()

model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'alpha' : [i for i in range(1,11)],
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('BernoulliNB: ')
print(model.best_params_)
print(model.best_score_)
#########################
algo = naive_bayes.MultinomialNB()

model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'alpha' : np.arange(1,11)
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('MultinomialNB: ')
print(model.best_params_)
print(model.best_score_)
##########################
algo = naive_bayes.GaussianNB()

model = model_selection.RandomizedSearchCV(algo, param_distributions = {
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print('GaussianNB: ')
print(model.best_params_)
print(model.best_score_)

BernoulliNB: 
{'alpha': 1}
0.8462585034013606
MultinomialNB: 
{'alpha': 1}
0.5108843537414967
GaussianNB: 
{}
0.8428571428571429


In [27]:
# apply Naive Bayes with gp minimize.

In [28]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = naive_bayes.BernoulliNB(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Integer(1,10, name = 'alpha')
    
]

param_names = ['alpha']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 10, n_random_starts = 10, verbose = 10)
print('BernoulliNB:')
print(dict(zip(param_names, result.x)))
print('###############################################################################')
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = naive_bayes.MultinomialNB(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Integer(1,10, name = 'alpha')
    
]

param_names = ['alpha']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 10, n_random_starts = 10, verbose = 10)
print('MultinomialNB:')
print(dict(zip(param_names, result.x)))


Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0251
Function value obtained: -0.8435
Current minimum: -0.8435
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0114
Function value obtained: -0.8435
Current minimum: -0.8435
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0081
Function value obtained: -0.8435
Current minimum: -0.8435
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0075
Function value obtained: -0.8456
Current minimum: -0.8456
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0075
Function value obtained: -0.8442
Current minimum: -0.8456
Iteration No: 6 started. 

In [29]:
# apply Naive Bayes with hyperopt.

In [30]:
def optimize(params, x, y):
    model = naive_bayes.BernoulliNB(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'alpha': scope.int(hp.quniform('alpha',1,10,1)),
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print('BernoulliNB: ')
print(result)
print('##################################################################################################')
def optimize(params, x, y):
    model = naive_bayes.MultinomialNB(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'alpha': scope.int(hp.quniform('alpha',1,10,1)),
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print('MultinomialNB')
print(result)

100%|██████████| 15/15 [00:00<00:00, 104.91trial/s, best loss: -0.845578231292517]
BernoulliNB: 
{'alpha': 9.0}
##################################################################################################
100%|██████████| 15/15 [00:00<00:00, 134.73trial/s, best loss: -0.5380952380952381]
MultinomialNB
{'alpha': 10.0}


In [31]:
# apply Naive Bayes with optuna.

In [32]:
def optimize(trial, x, y):
    alpha = trial.suggest_int('alpha', 1,10)
    model = naive_bayes.BernoulliNB(
        alpha = alpha
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 10)
print('BernoulliNB:')
print(study.best_params, study.best_value)
print('############################################################################################')
def optimize(trial, x, y):
    alpha = trial.suggest_int('alpha', 1,10)
    model = naive_bayes.MultinomialNB(
        alpha = alpha
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 10)
print('MultinomialNB:')
print(study.best_params, study.best_value)


[I 2022-08-13 09:21:56,612] A new study created in memory with name: no-name-0efef89c-c4ac-4976-8e2e-985668ed93ef
[I 2022-08-13 09:21:56,650] Trial 0 finished with value: 0.8428571428571429 and parameters: {'alpha': 4}. Best is trial 0 with value: 0.8428571428571429.
[I 2022-08-13 09:21:56,670] Trial 1 finished with value: 0.84421768707483 and parameters: {'alpha': 8}. Best is trial 1 with value: 0.84421768707483.
[I 2022-08-13 09:21:56,683] Trial 2 finished with value: 0.8435374149659864 and parameters: {'alpha': 6}. Best is trial 1 with value: 0.84421768707483.
[I 2022-08-13 09:21:56,695] Trial 3 finished with value: 0.8448979591836734 and parameters: {'alpha': 1}. Best is trial 3 with value: 0.8448979591836734.
[I 2022-08-13 09:21:56,709] Trial 4 finished with value: 0.8448979591836736 and parameters: {'alpha': 10}. Best is trial 4 with value: 0.8448979591836736.
[I 2022-08-13 09:21:56,723] Trial 5 finished with value: 0.8448979591836734 and parameters: {'alpha': 2}. Best is trial 4

BernoulliNB:
{'alpha': 10} 0.8448979591836736
############################################################################################
MultinomialNB:
{'alpha': 4} 0.5380952380952381


# Decision Tree Classifier with different hyper parameter tuning techniques.

In [33]:
# apply Decision Tree Classifier with GridSearch.

In [34]:
algo = tree.DecisionTreeClassifier()

model = model_selection.GridSearchCV(algo, param_grid = {
    'criterion': ['gini','entropy','log_loss'],
    'splitter': ['best','random'],
    'max_depth': [i for i in range(8,33)],
    "min_samples_leaf": [i for i in range(2,9)]
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'criterion': 'entropy', 'max_depth': 32, 'min_samples_leaf': 8, 'splitter': 'random'}
0.8510204081632653


In [35]:
# apply Decision Tree Classifier with RandomizedSearch.

In [36]:
algo = tree.DecisionTreeClassifier()
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'criterion': ['gini','entropy','log_loss'],
    'splitter': ['best','random'],
    'max_depth': np.arange(8,33),
    "min_samples_leaf": np.arange(2,9)
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'splitter': 'random', 'min_samples_leaf': 7, 'max_depth': 25, 'criterion': 'entropy'}
0.8380952380952381


In [37]:
# apply Decision Tree Classifier with gp minimize.

In [38]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = tree.DecisionTreeClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Categorical(['gini','entropy','log_loss'], name = 'criterion'),
    space.Categorical(['best','random'], name = 'splitter'),
    space.Integer(8,33, name = 'max_depth'),
    space.Integer(2,9, name = 'min_samples_leaf')
]

param_names = ['criterion','splitter','max_depth','min_samples_leaf']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0203
Function value obtained: -0.8109
Current minimum: -0.8109
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0137
Function value obtained: -0.8068
Current minimum: -0.8109
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0133
Function value obtained: -0.8252
Current minimum: -0.8252
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0149
Function value obtained: -0.8095
Current minimum: -0.8252
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0156
Function value obtained: -0.8122
Current minimum: -0.8252
Iteration No: 6 started. 

In [39]:
# apply Decision Tree Classifier with hyperopt.

In [40]:
def optimize(params, x, y):
    model = tree.DecisionTreeClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'criterion': hp.choice('criterion',['gini','entropy','log_loss']),
    'splitter': hp.choice('splitter',['best','random']),
    'max_depth': scope.int(hp.quniform('max_depth',8,33,1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf',2,9,1))
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:00<00:00, 32.11trial/s, best loss: -0.8353741496598641]
{'criterion': 0, 'max_depth': 18.0, 'min_samples_leaf': 9.0, 'splitter': 0}


In [41]:
# apply Decision Tree Classifier with optuna.

In [42]:
def optimize(trial, x, y):
    criterion = trial.suggest_categorical('criterion', ['gini','entropy','log_loss'])
    splitter = trial.suggest_categorical('splitter', ['best','random'])
    max_depth = trial.suggest_int('max_depth', 8, 32)
    min_samples_leaf = trial.suggest_float('min_samples_leaf',2,8)
    model = tree.DecisionTreeClassifier(
        criterion = criterion,
        splitter = splitter,
        max_depth = max_depth,
        min_samples_leaf = min_samples_leaf
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-13 09:23:09,680] A new study created in memory with name: no-name-75abf936-6208-4bbc-9bfd-7118b25fc5ea
[I 2022-08-13 09:23:09,708] Trial 0 finished with value: 0.8387755102040817 and parameters: {'criterion': 'gini', 'splitter': 'random', 'max_depth': 11, 'min_samples_leaf': 4.103779772534061}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:23:09,726] Trial 1 finished with value: 0.8387755102040817 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 22, 'min_samples_leaf': 4.148248527254861}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:23:09,737] Trial 2 finished with value: 0.8387755102040817 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 18, 'min_samples_leaf': 5.864821874100996}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:23:09,745] Trial 3 finished with value: 0.8387755102040817 and parameters: {'criterion': 'gini', 'splitter': 'random', 'max_depth': 9, 'min_sam

{'criterion': 'gini', 'splitter': 'random', 'max_depth': 11, 'min_samples_leaf': 4.103779772534061} 0.8387755102040817


# Random Forest Classifier with different hyper parameter tuning techniques.

In [43]:
# apply Randome Forest Classifier with GridSearch.

In [44]:
algo = ensemble.RandomForestClassifier()

model = model_selection.GridSearchCV(algo, param_grid = {
    'criterion': ['gini','entropy','log_loss'],
    'max_depth': [i for i in range(8,16)],
    "min_samples_leaf": [i for i in range(2,9)]
}, scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))
model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'criterion': 'entropy', 'max_depth': 14, 'min_samples_leaf': 2}
0.8517006802721088


In [45]:
# apply Randome Forest Classifier with RandomizedSearch.

In [46]:
algo = ensemble.RandomForestClassifier()
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'criterion': ['gini','entropy','log_loss'],
    'max_depth': np.arange(8,33),
    "min_samples_leaf": np.arange(2,9)
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'min_samples_leaf': 3, 'max_depth': 10, 'criterion': 'entropy'}
0.8544217687074831


In [47]:
# apply Random Forest Classifier with gp minimize.

In [48]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = ensemble.RandomForestClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Categorical(['gini','entropy','log_loss'], name = 'criterion'),
    space.Integer(100,250,name = 'n_estimators'),
    space.Integer(8,33, name = 'max_depth'),
    space.Integer(2,9, name = 'min_samples_leaf')
]

param_names = ['criterion','n_estimators','max_depth','min_samples_leaf']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.4036
Function value obtained: -0.8490
Current minimum: -0.8490
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.3745
Function value obtained: -0.8490
Current minimum: -0.8490
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.3091
Function value obtained: -0.8531
Current minimum: -0.8531
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.6302
Function value obtained: -0.8537
Current minimum: -0.8537
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 1.7410
Function value obtained: -0.8531
Current minimum: -0.8537
Iteration No: 6 started. 

In [49]:
# apply Random Forest Classifier with hyperopt.

In [50]:
def optimize(params, x, y):
    model = ensemble.RandomForestClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'criterion': hp.choice('criterion',['gini','entropy','log_loss']),
    'n_estimators': scope.int(hp.quniform('n_estimators',100,250,1)),
    'max_depth': scope.int(hp.quniform('max_depth',8,33,1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf',2,9,1))
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:25<00:00,  1.73s/trial, best loss: -0.8578231292517007]
{'criterion': 2, 'max_depth': 13.0, 'min_samples_leaf': 3.0, 'n_estimators': 153.0}


In [51]:
# apply Random Forest Classifier with optuna.

In [52]:
def optimize(trial, x, y):
    criterion = trial.suggest_categorical('criterion', ['gini','entropy','log_loss'])
    n_estimators = trial.suggest_int('n_estimators', 100,250)
    max_depth = trial.suggest_int('max_depth', 8, 32)
    min_samples_leaf = trial.suggest_float('min_samples_leaf',2,8)
    model = ensemble.RandomForestClassifier(
        criterion = criterion,
        n_estimators = n_estimators,
        max_depth = max_depth,
        min_samples_leaf = min_samples_leaf
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-13 09:30:35,193] A new study created in memory with name: no-name-a565fb4a-db4f-4b91-be1c-2efd40531b0f
[I 2022-08-13 09:30:36,474] Trial 0 finished with value: 0.8387755102040817 and parameters: {'criterion': 'gini', 'n_estimators': 240, 'max_depth': 23, 'min_samples_leaf': 3.5749820704403295}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:30:37,111] Trial 1 finished with value: 0.8387755102040817 and parameters: {'criterion': 'gini', 'n_estimators': 121, 'max_depth': 19, 'min_samples_leaf': 5.447620074737706}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:30:37,717] Trial 2 finished with value: 0.8387755102040817 and parameters: {'criterion': 'gini', 'n_estimators': 116, 'max_depth': 30, 'min_samples_leaf': 4.522590181904749}. Best is trial 0 with value: 0.8387755102040817.
[I 2022-08-13 09:30:39,004] Trial 3 finished with value: 0.8387755102040817 and parameters: {'criterion': 'gini', 'n_estimators': 249, 'max_depth': 28, 'min_samples_

{'criterion': 'gini', 'n_estimators': 240, 'max_depth': 23, 'min_samples_leaf': 3.5749820704403295} 0.8387755102040817


# AdaBoost Classifier with different hyper parameter tuning techniques.

In [53]:
# apply AdaBoost Classifier with GridSearch.

In [75]:
algo = ensemble.AdaBoostClassifier()
list = []
start, step, end = 0.01, 0.01, 0.11
while start < end:
    list.append(start)
    start += step
model = model_selection.GridSearchCV(algo, param_grid = {
    'learning_rate': list,
    'n_estimators': [i for i in range(50,100)],
    'algorithm': ['SAMME','SAMME.R']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'algorithm': 'SAMME.R', 'learning_rate': 0.10999999999999999, 'n_estimators': 99}
0.8564625850340135


In [ ]:
# apply AdaBoost Classifier with RandomizedSearch.

In [70]:
algo = ensemble.AdaBoostClassifier()
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'learning_rate': np.arange(0.01,0.11,0.01),
    'n_estimators': np.arange(50,101),
    'algorithm': ['SAMME','SAMME.R']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'n_estimators': 100, 'learning_rate': 0.09999999999999999, 'algorithm': 'SAMME.R'}
0.8530612244897959


In [ ]:
# apply AdaBoost Classifier with gp minimize.

In [55]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = ensemble.AdaBoostClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Real(0.01,0.11, name = 'learning_rate'),
    space.Integer(50,101, name = 'n_estimators'),
    space.Categorical(['SAMME','SAMME.R'], name = 'algorithm'),
]

param_names = ['learning_rate','n_estimators','algorithm']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.7252
Function value obtained: -0.8463
Current minimum: -0.8463
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.9299
Function value obtained: -0.8469
Current minimum: -0.8469
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.6082
Function value obtained: -0.8422
Current minimum: -0.8469
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.9092
Function value obtained: -0.8388
Current minimum: -0.8469
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 1.0611
Function value obtained: -0.8429
Current minimum: -0.8469
Iteration No: 6 started. 

In [ ]:
# apply AdaBoost Classifier with hyperopt.

In [56]:
def optimize(params, x, y):
    model = ensemble.AdaBoostClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'n_estimators': scope.int(hp.quniform('n_estimators',50,101,1)),
    'learning_rate': hp.loguniform('learning_rate',np.log(0.01),np.log(0.11)),
    'algorithm': hp.choice('algorithm',['SAMME','SAMME.R'])
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:11<00:00,  1.29trial/s, best loss: -0.84421768707483] 
{'algorithm': 1, 'learning_rate': 0.07662543711901312, 'n_estimators': 65.0}


In [ ]:
# apply AdaBoost Classifier with optuna.

In [57]:
def optimize(trial, x, y):
    learning_rate = trial.suggest_float('learning_rate',0.01,0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 150)
    algorithm = trial.suggest_categorical('algorithm', ['SAMME','SAMME.R'])
    model = ensemble.AdaBoostClassifier(
        learning_rate = learning_rate,
        n_estimators = n_estimators,
        algorithm = algorithm
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-13 10:09:52,432] A new study created in memory with name: no-name-e4f86e03-e88e-4039-90ad-c8f599ad8fe7
[I 2022-08-13 10:09:54,098] Trial 0 finished with value: 0.8408163265306123 and parameters: {'learning_rate': 0.03393234103842575, 'n_estimators': 143, 'algorithm': 'SAMME'}. Best is trial 0 with value: 0.8408163265306123.
[I 2022-08-13 10:09:55,467] Trial 1 finished with value: 0.8394557823129253 and parameters: {'learning_rate': 0.027010034089830236, 'n_estimators': 121, 'algorithm': 'SAMME.R'}. Best is trial 0 with value: 0.8408163265306123.
[I 2022-08-13 10:09:56,888] Trial 2 finished with value: 0.8421768707482993 and parameters: {'learning_rate': 0.0575035706919553, 'n_estimators': 145, 'algorithm': 'SAMME'}. Best is trial 2 with value: 0.8421768707482993.
[I 2022-08-13 10:09:57,777] Trial 3 finished with value: 0.8408163265306123 and parameters: {'learning_rate': 0.06964417775288392, 'n_estimators': 91, 'algorithm': 'SAMME'}. Best is trial 2 with value: 0.84217687074

{'learning_rate': 0.08097354128137506, 'n_estimators': 122, 'algorithm': 'SAMME.R'} 0.8489795918367348


# Gradient Boosting Classifier with different hyper parameter tuning techniques.

In [ ]:
# apply Gradient Boosting Classifier with GridSearch.

In [74]:
algo = ensemble.GradientBoostingClassifier()
list = []
start, step, end = 0.1, 0.01, 0.21
while start < end:
    list.append(start)
    start += step
model = model_selection.GridSearchCV(algo, param_grid = {
    'loss': ['log_loss', 'deviance', 'exponential'],
    'learning_rate': list,
    'criterion': ['friedman_mse', 'squared_error', 'mse']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'criterion': 'friedman_mse', 'learning_rate': 0.19000000000000006, 'loss': 'exponential'}
0.8700680272108844


In [ ]:
# apply Gradient Boosting Classifier with RandomizedSearch.

In [71]:
algo = ensemble.GradientBoostingClassifier()
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'loss': ['log_loss', 'deviance', 'exponential'],
    'learning_rate': np.arange(0.1,0.21,0.01),
    'criterion': ['friedman_mse', 'squared_error', 'mse']
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'loss': 'exponential', 'learning_rate': 0.16999999999999998, 'criterion': 'squared_error'}
0.8653061224489795


In [ ]:
# apply Gradient Boosting Classifier with gp minimize.

In [58]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = ensemble.GradientBoostingClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Categorical(['log_loss', 'deviance', 'exponential'], name = 'loss'),
    space.Integer(100,500, name = 'n_estimators'),
    space.Real(0.1,0.21,name = 'learning_rate'),
    space.Categorical(['friedman_mse', 'squared_error', 'mse'], name = 'criterion'),
    space.Integer(2,9, name = 'min_samples_leaf'),
    space.Integer(3,11, name = 'max_depth')
]

param_names = ['loss','n_estimators','learning_rate','criterion','min_samples_leaf','max_depth']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 9.6520
Function value obtained: -0.8605
Current minimum: -0.8605
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 7.2823
Function value obtained: -0.8626
Current minimum: -0.8626
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 15.2653
Function value obtained: -0.8585
Current minimum: -0.8626
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 9.6111
Function value obtained: -0.8619
Current minimum: -0.8626
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 7.9255
Function value obtained: -0.8626
Current minimum: -0.8626
Iteration No: 6 started.

In [ ]:
# apply Gradient Boosting Classifier with hyperopt.

In [59]:
def optimize(params, x, y):
    model = ensemble.GradientBoostingClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'loss': hp.choice('loss',['log_loss', 'deviance', 'exponential']),
    'n_estimators': scope.int(hp.quniform('n_estimators',100,500,1)),
    'learning_rate': hp.loguniform('learning_rate',np.log(0.1),np.log(0.2)),
    'criterion': hp.choice('criterion',['friedman_mse', 'squared_error', 'mse']),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf',2,9,1)),
    'max_depth': scope.int(hp.quniform('max_depth',3,11,1))
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [02:07<00:00,  8.47s/trial, best loss: -0.8673469387755102]
{'criterion': 1, 'learning_rate': 0.1742883503142632, 'loss': 1, 'max_depth': 5.0, 'min_samples_leaf': 5.0, 'n_estimators': 492.0}


In [ ]:
# apply Gradient Boosting Classifier with optuna.

In [60]:
def optimize(trial, x, y):
    loss = trial.suggest_categorical('loss',['log_loss', 'deviance', 'exponential'])
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)
    learning_rate = trial.suggest_float('learning_rate',0.1,0.2)
    criterion = trial.suggest_categorical('criterion',['friedman_mse', 'squared_error', 'mse'])
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 8)
    max_depth = trial.suggest_int('max_depth', 8, 32)
    model = ensemble.GradientBoostingClassifier(
        loss = loss,
        n_estimators = n_estimators,
        learning_rate = learning_rate,
        criterion = criterion,
        min_samples_leaf = min_samples_leaf,
        max_depth = max_depth
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-13 10:32:13,405] A new study created in memory with name: no-name-67b90679-9884-45e1-b02c-049a19048718
[I 2022-08-13 10:32:25,100] Trial 0 finished with value: 0.85578231292517 and parameters: {'loss': 'deviance', 'n_estimators': 495, 'learning_rate': 0.16899700865399653, 'criterion': 'squared_error', 'min_samples_leaf': 3, 'max_depth': 30}. Best is trial 0 with value: 0.85578231292517.
[I 2022-08-13 10:32:40,130] Trial 1 finished with value: 0.8585034013605443 and parameters: {'loss': 'log_loss', 'n_estimators': 460, 'learning_rate': 0.1083704589307928, 'criterion': 'squared_error', 'min_samples_leaf': 8, 'max_depth': 12}. Best is trial 1 with value: 0.8585034013605443.
[I 2022-08-13 10:32:49,402] Trial 2 finished with value: 0.8598639455782312 and parameters: {'loss': 'log_loss', 'n_estimators': 236, 'learning_rate': 0.13140192071875262, 'criterion': 'friedman_mse', 'min_samples_leaf': 8, 'max_depth': 9}. Best is trial 2 with value: 0.8598639455782312.
[I 2022-08-13 10:32:

{'loss': 'exponential', 'n_estimators': 252, 'learning_rate': 0.16181890873861313, 'criterion': 'squared_error', 'min_samples_leaf': 6, 'max_depth': 24} 0.8646258503401361


# XGB Classifier with different hyper parameter tuning techniques.

In [ ]:
# apply XGB Classifier with GridSearch.

In [73]:
algo = xgb.XGBClassifier()
list = []
start, step, end = 0.1, 0.01, 0.2
while start < end:
    list.append(start)
    start += step
model = model_selection.GridSearchCV(algo, param_grid = {
    'sampling_method': ['uniform','gradient_based'],
    'eta': list
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10))

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'eta': 0.18000000000000005, 'sampling_method': 'uniform'}
0.8768707482993197


In [ ]:
# apply XGB Classifier with RandomizedSearch.

In [72]:
algo = xgb.XGBClassifier()
model = model_selection.RandomizedSearchCV(algo, param_distributions = {
    'sampling_method': ['uniform','gradient_based'],
    'eta': np.arange(0.1,0.2,0.01)
},
scoring = 'accuracy', cv = model_selection.StratifiedShuffleSplit(n_splits = 10), n_iter = 15)

model.fit(X,y)
print(model.best_params_)
print(model.best_score_)

{'sampling_method': 'uniform', 'eta': 0.14999999999999997}
0.8564625850340135


In [ ]:
# apply XGB Classifier with gp minimize.

In [61]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = xgb.XGBClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Categorical(['uniform','gradient_based'], name = 'sampling_method'),
    space.Real(0.1,0.21,name = 'eta'),
    space.Integer(3,11, name = 'max_depth')
]

param_names = ['sampling_method','eta','max_depth']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 2.2853
Function value obtained: -0.8551
Current minimum: -0.8551
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.0180
Function value obtained: -0.8599
Current minimum: -0.8599
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.7796
Function value obtained: -0.8626
Current minimum: -0.8626
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 1.5809
Function value obtained: -0.8633
Current minimum: -0.8633
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 1.8079
Function value obtained: -0.8592
Current minimum: -0.8633
Iteration No: 6 started. 

In [ ]:
# apply XGB Classifier with hyperopt.

In [62]:
def optimize(params, x, y):
    model = xgb.XGBClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'sampling_method': hp.choice('sampling_method',['uniform','gradient_based']),
    'eta': hp.loguniform('eta',np.log(0.1),np.log(1)),
    'max_depth': scope.int(hp.quniform('max_depth',3,11,1))
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

100%|██████████| 15/15 [00:28<00:00,  1.90s/trial, best loss: -0.8646258503401361]
{'eta': 0.2396354093621062, 'max_depth': 6.0, 'sampling_method': 1}


In [ ]:
# apply XGB Classifier with optuna.

In [63]:
def optimize(trial, x, y):
    sampling_method = trial.suggest_categorical('sampling_method',['uniform','gradient_based'])
    eta = trial.suggest_float('eta',0.1,0.9)
    max_depth = trial.suggest_int('max_depth', 8, 32)
    model = xgb.XGBClassifier(
        sampling_method = sampling_method,
        eta = eta,
        max_depth = max_depth,
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-13 10:37:07,549] A new study created in memory with name: no-name-e87f5ef6-aeed-4798-a0b0-44ea101e2c4f
[I 2022-08-13 10:37:10,015] Trial 0 finished with value: 0.8530612244897959 and parameters: {'sampling_method': 'gradient_based', 'eta': 0.8554361602911039, 'max_depth': 9}. Best is trial 0 with value: 0.8530612244897959.
[I 2022-08-13 10:37:12,123] Trial 1 finished with value: 0.8571428571428571 and parameters: {'sampling_method': 'gradient_based', 'eta': 0.2447029112596825, 'max_depth': 8}. Best is trial 1 with value: 0.8571428571428571.
[I 2022-08-13 10:37:13,998] Trial 2 finished with value: 0.8585034013605443 and parameters: {'sampling_method': 'uniform', 'eta': 0.7920192913950916, 'max_depth': 11}. Best is trial 2 with value: 0.8585034013605443.
[I 2022-08-13 10:37:15,937] Trial 3 finished with value: 0.8591836734693878 and parameters: {'sampling_method': 'gradient_based', 'eta': 0.4017199144720338, 'max_depth': 11}. Best is trial 3 with value: 0.8591836734693878.
[I 

{'sampling_method': 'gradient_based', 'eta': 0.4437316246535864, 'max_depth': 31} 0.8639455782312926


# Cat Boost Classifier with different hyper parameter tuning techniques.

In [ ]:
# apply Cat Boost Classifier with gp minimize.

In [65]:
def optimize(params, params_names, x, y):
    params = dict(zip(params_names, params))
    model = catboost.CatBoostClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = [
    space.Real(0.01,0.11,name = 'learning_rate'),
    space.Integer(4,16, name = 'depth'),
    space.Integer(50,100, name = 'iterations'),
    space.Integer(1,5, name = 'l2_leaf_reg')
]

param_names = ['learning_rate','depth','iterations','l2_leaf_reg']

optimaization_function = partial(optimize, params_names = param_names, x = X, y = y)

result = gp_minimize(optimaization_function,dimensions = param_spaces, n_calls = 15, n_random_starts = 10, verbose = 10)
print(dict(zip(param_names, result.x)))

Iteration No: 1 started. Evaluating function at random point.
0:	learn: 0.6632671	total: 16s	remaining: 15m 58s
1:	learn: 0.6344812	total: 17.9s	remaining: 8m 48s
2:	learn: 0.6081664	total: 18.5s	remaining: 5m 58s
3:	learn: 0.5843812	total: 19.1s	remaining: 4m 32s
4:	learn: 0.5614447	total: 19.7s	remaining: 3m 40s
5:	learn: 0.5392947	total: 20.3s	remaining: 3m 5s
6:	learn: 0.5157873	total: 20.9s	remaining: 2m 41s
7:	learn: 0.4958474	total: 21.4s	remaining: 2m 22s
8:	learn: 0.4756003	total: 22s	remaining: 2m 7s
9:	learn: 0.4583281	total: 22.6s	remaining: 1m 55s
10:	learn: 0.4418166	total: 23.2s	remaining: 1m 45s
11:	learn: 0.4237793	total: 23.7s	remaining: 1m 36s
12:	learn: 0.4088446	total: 24s	remaining: 1m 28s
13:	learn: 0.3951660	total: 24.6s	remaining: 1m 22s
14:	learn: 0.3809093	total: 25.2s	remaining: 1m 17s
15:	learn: 0.3684618	total: 25.8s	remaining: 1m 12s
16:	learn: 0.3566329	total: 26.4s	remaining: 1m 8s
17:	learn: 0.3459682	total: 27.1s	remaining: 1m 4s
18:	learn: 0.3302636	

39:	learn: 0.1826012	total: 20.9s	remaining: 10.9s
40:	learn: 0.1786927	total: 21.4s	remaining: 10.4s
41:	learn: 0.1748581	total: 22s	remaining: 9.94s
42:	learn: 0.1711991	total: 22.5s	remaining: 9.44s
43:	learn: 0.1657936	total: 22.6s	remaining: 8.72s
44:	learn: 0.1620712	total: 23.2s	remaining: 8.23s
45:	learn: 0.1587354	total: 23.7s	remaining: 7.74s
46:	learn: 0.1554402	total: 24.3s	remaining: 7.23s
47:	learn: 0.1524825	total: 24.8s	remaining: 6.73s
48:	learn: 0.1487784	total: 25.4s	remaining: 6.23s
49:	learn: 0.1460428	total: 26s	remaining: 5.72s
50:	learn: 0.1433381	total: 26.6s	remaining: 5.21s
51:	learn: 0.1404315	total: 27.1s	remaining: 4.69s
52:	learn: 0.1378374	total: 27.7s	remaining: 4.18s
53:	learn: 0.1350555	total: 28.3s	remaining: 3.66s
54:	learn: 0.1325987	total: 28.9s	remaining: 3.15s
55:	learn: 0.1301895	total: 29.5s	remaining: 2.63s
56:	learn: 0.1279628	total: 30s	remaining: 2.11s
57:	learn: 0.1257445	total: 30.6s	remaining: 1.58s
58:	learn: 0.1234347	total: 31.2s	rem

55:	learn: 0.0672472	total: 273ms	remaining: 24.4ms
56:	learn: 0.0653841	total: 277ms	remaining: 19.5ms
57:	learn: 0.0643417	total: 282ms	remaining: 14.6ms
58:	learn: 0.0632322	total: 286ms	remaining: 9.71ms
59:	learn: 0.0618061	total: 291ms	remaining: 4.85ms
60:	learn: 0.0608765	total: 296ms	remaining: 0us
0:	learn: 0.6121709	total: 14.5ms	remaining: 869ms
1:	learn: 0.5448145	total: 26.6ms	remaining: 784ms
2:	learn: 0.4973286	total: 37.7ms	remaining: 729ms
3:	learn: 0.4572804	total: 48.5ms	remaining: 691ms
4:	learn: 0.4207360	total: 59.9ms	remaining: 671ms
5:	learn: 0.3824054	total: 72.4ms	remaining: 664ms
6:	learn: 0.3558578	total: 85.2ms	remaining: 657ms
7:	learn: 0.3404422	total: 96.2ms	remaining: 637ms
8:	learn: 0.3230404	total: 108ms	remaining: 626ms
9:	learn: 0.3102952	total: 120ms	remaining: 611ms
10:	learn: 0.2901177	total: 130ms	remaining: 593ms
11:	learn: 0.2763172	total: 142ms	remaining: 580ms
12:	learn: 0.2635727	total: 153ms	remaining: 566ms
13:	learn: 0.2513061	total: 16

54:	learn: 0.0741757	total: 410ms	remaining: 44.8ms
55:	learn: 0.0733025	total: 415ms	remaining: 37.1ms
56:	learn: 0.0713596	total: 420ms	remaining: 29.5ms
57:	learn: 0.0699048	total: 425ms	remaining: 22ms
58:	learn: 0.0687857	total: 429ms	remaining: 14.6ms
59:	learn: 0.0670982	total: 434ms	remaining: 7.24ms
60:	learn: 0.0653032	total: 440ms	remaining: 0us
0:	learn: 0.6182372	total: 11.7ms	remaining: 701ms
1:	learn: 0.5519892	total: 26.3ms	remaining: 777ms
2:	learn: 0.5094934	total: 40ms	remaining: 774ms
3:	learn: 0.4714855	total: 53.5ms	remaining: 763ms
4:	learn: 0.4431352	total: 66.3ms	remaining: 742ms
5:	learn: 0.4044961	total: 79.5ms	remaining: 729ms
6:	learn: 0.3752717	total: 92.7ms	remaining: 715ms
7:	learn: 0.3469473	total: 107ms	remaining: 711ms
8:	learn: 0.3278426	total: 121ms	remaining: 698ms
9:	learn: 0.3092966	total: 136ms	remaining: 692ms
10:	learn: 0.2929576	total: 148ms	remaining: 673ms
11:	learn: 0.2756101	total: 161ms	remaining: 656ms
12:	learn: 0.2616036	total: 173ms	

90:	learn: 0.0399489	total: 25.2s	remaining: 1.94s
91:	learn: 0.0394924	total: 25.5s	remaining: 1.66s
92:	learn: 0.0390517	total: 25.8s	remaining: 1.39s
93:	learn: 0.0386178	total: 26s	remaining: 1.11s
94:	learn: 0.0381922	total: 26.3s	remaining: 831ms
95:	learn: 0.0377823	total: 26.6s	remaining: 555ms
96:	learn: 0.0373832	total: 27s	remaining: 278ms
97:	learn: 0.0369865	total: 27.2s	remaining: 0us
0:	learn: 0.6071990	total: 9.41ms	remaining: 913ms
1:	learn: 0.5617702	total: 294ms	remaining: 14.1s
2:	learn: 0.5100940	total: 576ms	remaining: 18.2s
3:	learn: 0.4742487	total: 577ms	remaining: 13.5s
4:	learn: 0.4444238	total: 871ms	remaining: 16.2s
5:	learn: 0.4161480	total: 1.19s	remaining: 18.3s
6:	learn: 0.3870137	total: 1.48s	remaining: 19.2s
7:	learn: 0.3623600	total: 1.8s	remaining: 20.3s
8:	learn: 0.3402419	total: 2.11s	remaining: 20.9s
9:	learn: 0.3205066	total: 2.39s	remaining: 21s
10:	learn: 0.3021516	total: 2.67s	remaining: 21.1s
11:	learn: 0.2831273	total: 2.95s	remaining: 21.1

...

0:	learn: 0.6864746	total: 2.85ms	remaining: 259ms
1:	learn: 0.6807777	total: 6.36ms	remaining: 286ms
2:	learn: 0.6750314	total: 9.28ms	remaining: 275ms
3:	learn: 0.6691813	total: 12.2ms	remaining: 268ms
4:	learn: 0.6627667	total: 15ms	remaining: 261ms
5:	learn: 0.6563838	total: 18ms	remaining: 258ms
6:	learn: 0.6507223	total: 21.3ms	remaining: 259ms
7:	learn: 0.6448697	total: 24.5ms	remaining: 257ms
8:	learn: 0.6398058	total: 27.7ms	remaining: 256ms
9:	learn: 0.6338400	total: 30.8ms	remaining: 253ms
10:	learn: 0.6275952	total: 33.9ms	remaining: 250ms
11:	learn: 0.6221130	total: 35.8ms	remaining: 238ms
12:	learn: 0.6162712	total: 37.6ms	remaining: 228ms
13:	learn: 0.6108065	total: 39.4ms	remaining: 219ms
14:	learn: 0.6059765	total: 41.2ms	remaining: 211ms
15:	learn: 0.6017288	total: 43ms	remaining: 204ms
16:	learn: 0.5974639	total: 44.8ms	remaining: 198ms
17:	learn: 0.5933682	total: 46.8ms	remaining: 192ms
18:	learn: 0.5881600	total: 48.6ms	remaining: 187ms
19:	learn: 0.5842319	total: 

0:	learn: 0.6395192	total: 3.12ms	remaining: 296ms
1:	learn: 0.6017916	total: 6.42ms	remaining: 302ms
2:	learn: 0.5686658	total: 9.31ms	remaining: 289ms
3:	learn: 0.5380895	total: 12.2ms	remaining: 281ms
4:	learn: 0.5084441	total: 15.5ms	remaining: 282ms
5:	learn: 0.4826972	total: 18.7ms	remaining: 281ms
6:	learn: 0.4585628	total: 21.5ms	remaining: 274ms
7:	learn: 0.4433701	total: 24.7ms	remaining: 272ms
8:	learn: 0.4269416	total: 27.6ms	remaining: 267ms
9:	learn: 0.4113674	total: 30.6ms	remaining: 263ms
10:	learn: 0.4033606	total: 33.9ms	remaining: 262ms
11:	learn: 0.3942479	total: 36.9ms	remaining: 258ms
12:	learn: 0.3877278	total: 39.9ms	remaining: 255ms
13:	learn: 0.3806086	total: 42.8ms	remaining: 251ms
14:	learn: 0.3745725	total: 46.1ms	remaining: 249ms
15:	learn: 0.3705501	total: 49.3ms	remaining: 247ms
16:	learn: 0.3661910	total: 52.6ms	remaining: 244ms
17:	learn: 0.3597426	total: 55.9ms	remaining: 242ms
18:	learn: 0.3551069	total: 59.1ms	remaining: 239ms
19:	learn: 0.3507561	t

0:	learn: 0.6393837	total: 2.87ms	remaining: 273ms
1:	learn: 0.6015140	total: 6.2ms	remaining: 291ms
2:	learn: 0.5613805	total: 9.08ms	remaining: 282ms
3:	learn: 0.5267917	total: 12.2ms	remaining: 281ms
4:	learn: 0.4992985	total: 15.2ms	remaining: 278ms
5:	learn: 0.4776920	total: 18.1ms	remaining: 271ms
6:	learn: 0.4617227	total: 21.3ms	remaining: 271ms
7:	learn: 0.4462412	total: 24.1ms	remaining: 266ms
8:	learn: 0.4277605	total: 27ms	remaining: 261ms
9:	learn: 0.4117925	total: 30.1ms	remaining: 259ms
10:	learn: 0.4012571	total: 33.3ms	remaining: 257ms
11:	learn: 0.3915423	total: 36.3ms	remaining: 254ms
12:	learn: 0.3816180	total: 39.4ms	remaining: 252ms
13:	learn: 0.3745358	total: 42.4ms	remaining: 248ms
14:	learn: 0.3687230	total: 45.5ms	remaining: 246ms
15:	learn: 0.3635953	total: 48.4ms	remaining: 242ms
16:	learn: 0.3580694	total: 51.6ms	remaining: 240ms
17:	learn: 0.3547176	total: 54.4ms	remaining: 236ms
18:	learn: 0.3515918	total: 57.6ms	remaining: 233ms
19:	learn: 0.3459280	tota

Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 4.7769
Function value obtained: -0.8687
Current minimum: -0.8687
Iteration No: 15 started. Searching for the next optimal point.
0:	learn: 0.6561649	total: 654us	remaining: 32.7ms
1:	learn: 0.6255089	total: 1.38ms	remaining: 33.9ms
2:	learn: 0.5975882	total: 1.97ms	remaining: 31.5ms
3:	learn: 0.5704016	total: 2.69ms	remaining: 31.7ms
4:	learn: 0.5449831	total: 3.38ms	remaining: 31.1ms
5:	learn: 0.5221251	total: 4.01ms	remaining: 30.1ms
6:	learn: 0.5049018	total: 4.7ms	remaining: 29.6ms
7:	learn: 0.4891222	total: 5.36ms	remaining: 28.8ms
8:	learn: 0.4756747	total: 6ms	remaining: 28ms
9:	learn: 0.4602302	total: 6.7ms	remaining: 27.5ms
10:	learn: 0.4446742	total: 7.32ms	remaining: 26.6ms
11:	learn: 0.4343584	total: 7.96ms	remaining: 25.9ms
12:	learn: 0.4214272	total: 8.62ms	remaining: 25.2ms
13:	learn: 0.4115772	total: 9.3ms	remaining: 24.6ms
14:	learn: 0.4038867	total: 9.94ms	remaining: 23.9ms
15:	learn: 0.39

0:	learn: 0.6528480	total: 3.1ms	remaining: 155ms
1:	learn: 0.6225680	total: 6.1ms	remaining: 149ms
2:	learn: 0.5953946	total: 9.41ms	remaining: 151ms
3:	learn: 0.5687242	total: 12.6ms	remaining: 148ms
4:	learn: 0.5390010	total: 16.8ms	remaining: 155ms
5:	learn: 0.5172662	total: 19.7ms	remaining: 148ms
6:	learn: 0.5009706	total: 22.6ms	remaining: 142ms
7:	learn: 0.4859065	total: 25.5ms	remaining: 137ms
8:	learn: 0.4673836	total: 28.3ms	remaining: 132ms
9:	learn: 0.4550607	total: 31.3ms	remaining: 128ms
10:	learn: 0.4431863	total: 34.2ms	remaining: 124ms
11:	learn: 0.4324562	total: 37.1ms	remaining: 121ms
12:	learn: 0.4212499	total: 40.1ms	remaining: 117ms
13:	learn: 0.4121123	total: 43ms	remaining: 114ms
14:	learn: 0.4044927	total: 45.8ms	remaining: 110ms
15:	learn: 0.4000991	total: 49ms	remaining: 107ms
16:	learn: 0.3959001	total: 52.2ms	remaining: 104ms
17:	learn: 0.3916442	total: 55.3ms	remaining: 101ms
18:	learn: 0.3855933	total: 58.2ms	remaining: 98ms
19:	learn: 0.3814867	total: 6

In [ ]:
# apply Cat Boost Classifier with hyperopt.

In [64]:
def optimize(params, x, y):
    model = catboost.CatBoostClassifier(**params)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return -1.0 * np.mean(accuracies)

param_spaces = {
    'learning_rate':hp.loguniform('learning_rate',np.log(0.01),np.log(0.11)),
    'depth': scope.int(hp.quniform('depth',3,11,1)),
    'iterations': scope.int(hp.quniform('iterations',50,100,1)),
    'l2_leaf_reg': scope.int(hp.quniform('l2_leaf_reg',1,5,1))
}

optimaization_function = partial(optimize,x = X, y = y)

result = fmin(fn = optimaization_function,space = param_spaces, max_evals = 15, trials = Trials(), algo = tpe.suggest)
print(result)

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]0:	learn: 0.6601682	total: 63ms	remaining: 4.16s
1:	learn: 0.6386406	total: 66.1ms	remaining: 2.15s
2:	learn: 0.6126491	total: 70.6ms	remaining: 1.5s
3:	learn: 0.5858339	total: 77.6ms	remaining: 1.22s
4:	learn: 0.5590322	total: 82.6ms	remaining: 1.02s
5:	learn: 0.5371590	total: 86.2ms	remaining: 876ms
6:	learn: 0.5164368	total: 89.5ms	remaining: 767ms
7:	learn: 0.4993209	total: 91.8ms	remaining: 677ms
8:	learn: 0.4843967	total: 94.1ms	remaining: 607ms
9:	learn: 0.4723394	total: 96.3ms	remaining: 549ms
10:	learn: 0.4558765	total: 98.3ms	remaining: 501ms
11:	learn: 0.4457561	total: 100ms	remaining: 459ms
12:	learn: 0.4374654	total: 102ms	remaining: 423ms
13:	learn: 0.4266296	total: 104ms	remaining: 394ms
14:	learn: 0.4171460	total: 106ms	remaining: 366ms
15:	learn: 0.4064433	total: 107ms	remaining: 341ms
16:	learn: 0.3965550	total: 109ms	remaining: 319ms
17:	learn: 0.3861678	total: 110ms	remaining: 300ms
18:	learn: 0.3773770	total: 11

0:	learn: 0.6598485	total: 6.77ms	remaining: 447ms
1:	learn: 0.6390709	total: 9.8ms	remaining: 319ms
2:	learn: 0.6083567	total: 16.4ms	remaining: 349ms
3:	learn: 0.5813762	total: 23.3ms	remaining: 367ms
4:	learn: 0.5605137	total: 29.9ms	remaining: 371ms
5:	learn: 0.5427102	total: 36ms	remaining: 366ms
6:	learn: 0.5213728	total: 42.5ms	remaining: 365ms
7:	learn: 0.5012285	total: 49.3ms	remaining: 363ms
8:	learn: 0.4848358	total: 56.3ms	remaining: 363ms
9:	learn: 0.4671031	total: 64.3ms	remaining: 367ms
10:	learn: 0.4544173	total: 72ms	remaining: 367ms
11:	learn: 0.4427668	total: 79.4ms	remaining: 364ms
12:	learn: 0.4299248	total: 87.4ms	remaining: 363ms
13:	learn: 0.4177176	total: 94.7ms	remaining: 359ms
14:	learn: 0.4049770	total: 102ms	remaining: 353ms
15:	learn: 0.3955151	total: 106ms	remaining: 339ms
16:	learn: 0.3863359	total: 111ms	remaining: 328ms
17:	learn: 0.3791175	total: 115ms	remaining: 312ms
18:	learn: 0.3714649	total: 118ms	remaining: 297ms
19:	learn: 0.3638727	total: 121m

69:	learn: 0.1951235	total: 91.3ms	remaining: 1.3ms
70:	learn: 0.1932050	total: 92.5ms	remaining: 0us
0:	learn: 0.6184547	total: 1.13ms	remaining: 79.1ms
1:	learn: 0.5590994	total: 2.41ms	remaining: 83.1ms
2:	learn: 0.5087320	total: 3.63ms	remaining: 82.3ms
3:	learn: 0.4825940	total: 4.93ms	remaining: 82.5ms
4:	learn: 0.4509385	total: 6.03ms	remaining: 79.5ms
5:	learn: 0.4294556	total: 7.24ms	remaining: 78.4ms
6:	learn: 0.4121855	total: 8.37ms	remaining: 76.6ms
7:	learn: 0.3979919	total: 9.62ms	remaining: 75.8ms
8:	learn: 0.3876525	total: 10.9ms	remaining: 75.1ms
9:	learn: 0.3750276	total: 12ms	remaining: 73ms
10:	learn: 0.3670163	total: 13.2ms	remaining: 71.8ms
11:	learn: 0.3564109	total: 14.3ms	remaining: 70.1ms
12:	learn: 0.3475572	total: 15.5ms	remaining: 69.2ms
13:	learn: 0.3411873	total: 16.7ms	remaining: 68ms
14:	learn: 0.3343839	total: 17.9ms	remaining: 67ms
15:	learn: 0.3295483	total: 19.1ms	remaining: 65.8ms
16:	learn: 0.3246757	total: 20.2ms	remaining: 64.3ms
17:	learn: 0.31

0:	learn: 0.6177890	total: 1.73ms	remaining: 121ms
1:	learn: 0.5588768	total: 3.84ms	remaining: 133ms
2:	learn: 0.5083987	total: 5.97ms	remaining: 135ms
3:	learn: 0.4823295	total: 8.16ms	remaining: 137ms
4:	learn: 0.4530518	total: 10.7ms	remaining: 141ms
5:	learn: 0.4315315	total: 13ms	remaining: 141ms
6:	learn: 0.4131077	total: 15.4ms	remaining: 141ms
7:	learn: 0.3979615	total: 17.6ms	remaining: 138ms
8:	learn: 0.3875462	total: 19.8ms	remaining: 136ms
9:	learn: 0.3761245	total: 22.1ms	remaining: 135ms
10:	learn: 0.3690084	total: 24.4ms	remaining: 133ms
11:	learn: 0.3579944	total: 26.7ms	remaining: 131ms
12:	learn: 0.3505513	total: 28.3ms	remaining: 126ms
13:	learn: 0.3453818	total: 30.1ms	remaining: 122ms
14:	learn: 0.3407690	total: 31.7ms	remaining: 118ms
15:	learn: 0.3324739	total: 33.1ms	remaining: 114ms
16:	learn: 0.3282138	total: 34.5ms	remaining: 110ms
17:	learn: 0.3227077	total: 35.8ms	remaining: 105ms
18:	learn: 0.3170252	total: 37.2ms	remaining: 102ms
19:	learn: 0.3135982	tot

0:	learn: 0.6769359	total: 2.43ms	remaining: 119ms
1:	learn: 0.6614962	total: 5.36ms	remaining: 129ms
2:	learn: 0.6483795	total: 8.1ms	remaining: 127ms
3:	learn: 0.6345029	total: 10.4ms	remaining: 120ms
4:	learn: 0.6225827	total: 12.8ms	remaining: 115ms
5:	learn: 0.6120730	total: 15.3ms	remaining: 112ms
6:	learn: 0.6003725	total: 17.8ms	remaining: 109ms
7:	learn: 0.5892554	total: 20.1ms	remaining: 105ms
8:	learn: 0.5803504	total: 22.7ms	remaining: 103ms
9:	learn: 0.5717510	total: 25.4ms	remaining: 102ms
10:	learn: 0.5616318	total: 27ms	remaining: 95.8ms
11:	learn: 0.5511619	total: 28.6ms	remaining: 90.7ms
12:	learn: 0.5412423	total: 30ms	remaining: 85.4ms
13:	learn: 0.5347926	total: 31.1ms	remaining: 80ms
14:	learn: 0.5272779	total: 32.4ms	remaining: 75.5ms
15:	learn: 0.5208304	total: 33.5ms	remaining: 71.2ms
16:	learn: 0.5122857	total: 34.5ms	remaining: 67ms
17:	learn: 0.5044675	total: 35.5ms	remaining: 63.1ms
18:	learn: 0.4985503	total: 36.4ms	remaining: 59.4ms
19:	learn: 0.4916677	t

...

 87%|████████▋ | 13/15 [01:34<00:19,  9.97s/trial, best loss: -0.8673469387755102]0:	learn: 0.6237260	total: 11.2ms	remaining: 562ms
1:	learn: 0.5653829	total: 16.9ms	remaining: 414ms
2:	learn: 0.5218816	total: 23.7ms	remaining: 379ms
3:	learn: 0.4877627	total: 29.9ms	remaining: 351ms
4:	learn: 0.4522286	total: 35.5ms	remaining: 326ms
5:	learn: 0.4139100	total: 42.5ms	remaining: 319ms
6:	learn: 0.3857638	total: 48.5ms	remaining: 305ms
7:	learn: 0.3661710	total: 56.1ms	remaining: 301ms
8:	learn: 0.3511588	total: 62.5ms	remaining: 292ms
9:	learn: 0.3329290	total: 68.3ms	remaining: 280ms
10:	learn: 0.3147713	total: 74.9ms	remaining: 272ms
11:	learn: 0.2947529	total: 80.8ms	remaining: 263ms
12:	learn: 0.2795258	total: 87.7ms	remaining: 256ms
13:	learn: 0.2652383	total: 93.2ms	remaining: 246ms
14:	learn: 0.2545144	total: 98.5ms	remaining: 236ms
15:	learn: 0.2429461	total: 105ms	remaining: 229ms
16:	learn: 0.2304782	total: 112ms	remaining: 224ms
17:	learn: 0.2213152	total: 117ms	remaining: 2

35:	learn: 0.1338428	total: 210ms	remaining: 87.5ms
36:	learn: 0.1297353	total: 216ms	remaining: 81.7ms
37:	learn: 0.1261416	total: 222ms	remaining: 75.9ms
38:	learn: 0.1232782	total: 227ms	remaining: 69.8ms
39:	learn: 0.1187705	total: 234ms	remaining: 64.2ms
40:	learn: 0.1158186	total: 239ms	remaining: 58.4ms
41:	learn: 0.1115298	total: 245ms	remaining: 52.4ms
42:	learn: 0.1087943	total: 250ms	remaining: 46.6ms
43:	learn: 0.1057640	total: 255ms	remaining: 40.6ms
44:	learn: 0.1029530	total: 261ms	remaining: 34.8ms
45:	learn: 0.1001067	total: 267ms	remaining: 29ms
46:	learn: 0.0982021	total: 272ms	remaining: 23.2ms
47:	learn: 0.0962358	total: 278ms	remaining: 17.4ms
48:	learn: 0.0933434	total: 284ms	remaining: 11.6ms
49:	learn: 0.0914263	total: 289ms	remaining: 5.79ms
50:	learn: 0.0886568	total: 295ms	remaining: 0us
0:	learn: 0.6262337	total: 14ms	remaining: 698ms
1:	learn: 0.5628171	total: 22.2ms	remaining: 545ms
2:	learn: 0.5226518	total: 29.6ms	remaining: 473ms
3:	learn: 0.4896550	to

0:	learn: 0.6750338	total: 9.09ms	remaining: 827ms
1:	learn: 0.6609140	total: 15.8ms	remaining: 709ms
2:	learn: 0.6480700	total: 24.4ms	remaining: 723ms
3:	learn: 0.6346379	total: 32.9ms	remaining: 724ms
4:	learn: 0.6236607	total: 41.4ms	remaining: 721ms
5:	learn: 0.6087169	total: 49.7ms	remaining: 713ms
6:	learn: 0.5949862	total: 57.7ms	remaining: 701ms
7:	learn: 0.5805001	total: 65.9ms	remaining: 692ms
8:	learn: 0.5698573	total: 75.3ms	remaining: 695ms
9:	learn: 0.5594353	total: 84.4ms	remaining: 692ms
10:	learn: 0.5503624	total: 92.7ms	remaining: 682ms
11:	learn: 0.5398220	total: 101ms	remaining: 674ms
12:	learn: 0.5266157	total: 110ms	remaining: 669ms
13:	learn: 0.5183308	total: 120ms	remaining: 669ms
14:	learn: 0.5094336	total: 130ms	remaining: 666ms
15:	learn: 0.5009530	total: 139ms	remaining: 659ms
16:	learn: 0.4919419	total: 146ms	remaining: 643ms
17:	learn: 0.4855214	total: 148ms	remaining: 610ms
18:	learn: 0.4769678	total: 153ms	remaining: 589ms
19:	learn: 0.4664781	total: 15

74:	learn: 0.2446484	total: 200ms	remaining: 45.3ms
75:	learn: 0.2427174	total: 203ms	remaining: 42.6ms
76:	learn: 0.2410885	total: 205ms	remaining: 39.9ms
77:	learn: 0.2386839	total: 207ms	remaining: 37.2ms
78:	learn: 0.2370330	total: 209ms	remaining: 34.5ms
79:	learn: 0.2354926	total: 212ms	remaining: 31.8ms
80:	learn: 0.2337685	total: 214ms	remaining: 29.1ms
81:	learn: 0.2324197	total: 216ms	remaining: 26.4ms
82:	learn: 0.2302109	total: 218ms	remaining: 23.7ms
83:	learn: 0.2281339	total: 221ms	remaining: 21ms
84:	learn: 0.2265308	total: 223ms	remaining: 18.4ms
85:	learn: 0.2243773	total: 225ms	remaining: 15.7ms
86:	learn: 0.2223929	total: 227ms	remaining: 13.1ms
87:	learn: 0.2205055	total: 230ms	remaining: 10.4ms
88:	learn: 0.2185900	total: 232ms	remaining: 7.82ms
89:	learn: 0.2172845	total: 234ms	remaining: 5.21ms
90:	learn: 0.2159420	total: 237ms	remaining: 2.6ms
91:	learn: 0.2147042	total: 239ms	remaining: 0us
0:	learn: 0.6751426	total: 9.62ms	remaining: 876ms
1:	learn: 0.6601852

100%|██████████| 15/15 [01:53<00:00,  7.60s/trial, best loss: -0.8673469387755102]
{'depth': 5.0, 'iterations': 71.0, 'l2_leaf_reg': 5.0, 'learning_rate': 0.10711294370470735}


In [ ]:
# apply Cat Boost Classifier with optuna.

In [66]:
def optimize(trial, x, y):
    learning_rate = trial.suggest_float('learning_rate',0.01,0.1)
    depth = trial.suggest_int('depth', 4, 16)
    iterations = trial.suggest_int('iterations', 50, 100)
    l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 1, 5)
    model = catboost.CatBoostClassifier(
        learning_rate = learning_rate,
        depth = depth,
        iterations = iterations,
        l2_leaf_reg = l2_leaf_reg
    )

    skf = model_selection.StratifiedKFold(n_splits = 5)

    accuracies = []
    for idx in skf.split(X = x, y = y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        pred = model.predict(xtest)

        accuracies.append((metrics.accuracy_score(ytest, pred)))

    return np.mean(accuracies)

optimaization_function = partial(optimize,x = X, y = y)

study = optuna.create_study(direction = 'maximize')
study.optimize(optimaization_function, n_trials = 15)
print(study.best_params, study.best_value)

[I 2022-08-13 10:54:05,173] A new study created in memory with name: no-name-2c71eccc-6d87-455e-ad55-12dc9bf4310e


0:	learn: 0.6693737	total: 22.7ms	remaining: 1.93s
1:	learn: 0.6457085	total: 44.4ms	remaining: 1.86s
2:	learn: 0.6329929	total: 47.9ms	remaining: 1.32s
3:	learn: 0.6166309	total: 60.6ms	remaining: 1.24s
4:	learn: 0.5985039	total: 82.9ms	remaining: 1.34s
5:	learn: 0.5793077	total: 104ms	remaining: 1.38s
6:	learn: 0.5596363	total: 124ms	remaining: 1.4s
7:	learn: 0.5454503	total: 146ms	remaining: 1.42s
8:	learn: 0.5338826	total: 169ms	remaining: 1.44s
9:	learn: 0.5195472	total: 188ms	remaining: 1.43s
10:	learn: 0.5031114	total: 207ms	remaining: 1.41s
11:	learn: 0.4888265	total: 229ms	remaining: 1.41s
12:	learn: 0.4824811	total: 233ms	remaining: 1.3s
13:	learn: 0.4693015	total: 256ms	remaining: 1.32s
14:	learn: 0.4551241	total: 279ms	remaining: 1.32s
15:	learn: 0.4417633	total: 307ms	remaining: 1.34s
16:	learn: 0.4363650	total: 310ms	remaining: 1.26s
17:	learn: 0.4267251	total: 327ms	remaining: 1.23s
18:	learn: 0.4149972	total: 337ms	remaining: 1.19s
19:	learn: 0.4024244	total: 349ms	rema

75:	learn: 0.1348604	total: 614ms	remaining: 80.7ms
76:	learn: 0.1329494	total: 622ms	remaining: 72.7ms
77:	learn: 0.1320693	total: 630ms	remaining: 64.6ms
78:	learn: 0.1302019	total: 639ms	remaining: 56.6ms
79:	learn: 0.1283371	total: 647ms	remaining: 48.5ms
80:	learn: 0.1260859	total: 656ms	remaining: 40.5ms
81:	learn: 0.1244147	total: 664ms	remaining: 32.4ms
82:	learn: 0.1228875	total: 671ms	remaining: 24.3ms
83:	learn: 0.1210518	total: 679ms	remaining: 16.2ms
84:	learn: 0.1203334	total: 687ms	remaining: 8.08ms
85:	learn: 0.1192638	total: 695ms	remaining: 0us
0:	learn: 0.6703412	total: 15.8ms	remaining: 1.34s
1:	learn: 0.6460328	total: 26.4ms	remaining: 1.11s
2:	learn: 0.6335197	total: 27.6ms	remaining: 763ms
3:	learn: 0.6182688	total: 31.7ms	remaining: 650ms
4:	learn: 0.6009963	total: 39.1ms	remaining: 633ms
5:	learn: 0.5796975	total: 46.4ms	remaining: 619ms
6:	learn: 0.5604044	total: 54.1ms	remaining: 611ms
7:	learn: 0.5468146	total: 61.5ms	remaining: 600ms
8:	learn: 0.5328104	tot

80:	learn: 0.1318571	total: 616ms	remaining: 38ms
81:	learn: 0.1303357	total: 624ms	remaining: 30.4ms
82:	learn: 0.1285575	total: 633ms	remaining: 22.9ms
83:	learn: 0.1272976	total: 641ms	remaining: 15.3ms
84:	learn: 0.1255574	total: 651ms	remaining: 7.66ms
85:	learn: 0.1237761	total: 660ms	remaining: 0us
0:	learn: 0.6707756	total: 11.8ms	remaining: 1s
1:	learn: 0.6547247	total: 13.1ms	remaining: 549ms
2:	learn: 0.6395647	total: 21.5ms	remaining: 594ms
3:	learn: 0.6169764	total: 30ms	remaining: 614ms
4:	learn: 0.5979576	total: 38.4ms	remaining: 622ms
5:	learn: 0.5769258	total: 46.5ms	remaining: 621ms
6:	learn: 0.5563400	total: 53.4ms	remaining: 602ms
7:	learn: 0.5395345	total: 60.1ms	remaining: 586ms
8:	learn: 0.5235506	total: 67.9ms	remaining: 581ms
9:	learn: 0.5077895	total: 76ms	remaining: 577ms
10:	learn: 0.4919506	total: 82.5ms	remaining: 562ms
11:	learn: 0.4799401	total: 90.6ms	remaining: 558ms
12:	learn: 0.4741183	total: 91.2ms	remaining: 512ms
13:	learn: 0.4611065	total: 98.5ms

[I 2022-08-13 10:54:20,135] Trial 0 finished with value: 0.8537414965986395 and parameters: {'learning_rate': 0.0231454534391292, 'depth': 10, 'iterations': 86, 'l2_leaf_reg': 1}. Best is trial 0 with value: 0.8537414965986395.


...

[I 2022-08-13 11:02:05,611] Trial 13 finished with value: 0.8646258503401361 and parameters: {'learning_rate': 0.08474732033435947, 'depth': 6, 'iterations': 59, 'l2_leaf_reg': 2}. Best is trial 2 with value: 0.8673469387755102.


0:	learn: 0.6249851	total: 2.77ms	remaining: 164ms
1:	learn: 0.5834422	total: 4.12ms	remaining: 119ms
2:	learn: 0.5307224	total: 7.24ms	remaining: 138ms
3:	learn: 0.4878744	total: 10.3ms	remaining: 145ms
4:	learn: 0.4552051	total: 13.3ms	remaining: 146ms
5:	learn: 0.4329217	total: 16.3ms	remaining: 146ms
6:	learn: 0.4142388	total: 17.9ms	remaining: 136ms
7:	learn: 0.3952846	total: 20.9ms	remaining: 136ms
8:	learn: 0.3798418	total: 23.8ms	remaining: 135ms
9:	learn: 0.3658577	total: 26.7ms	remaining: 134ms
10:	learn: 0.3528282	total: 29.6ms	remaining: 132ms
11:	learn: 0.3413429	total: 32.7ms	remaining: 131ms
12:	learn: 0.3311036	total: 35.5ms	remaining: 128ms
13:	learn: 0.3205842	total: 38.5ms	remaining: 127ms
14:	learn: 0.3110195	total: 41.4ms	remaining: 124ms
15:	learn: 0.3038553	total: 44.3ms	remaining: 122ms
16:	learn: 0.2967308	total: 47.2ms	remaining: 119ms
17:	learn: 0.2892152	total: 50.1ms	remaining: 117ms
18:	learn: 0.2836273	total: 53.1ms	remaining: 115ms
19:	learn: 0.2775803	t

0:	learn: 0.6228619	total: 6.04ms	remaining: 356ms
1:	learn: 0.5830871	total: 9.08ms	remaining: 263ms
2:	learn: 0.5322883	total: 15.1ms	remaining: 287ms
3:	learn: 0.4909982	total: 21.1ms	remaining: 295ms
4:	learn: 0.4583654	total: 27.5ms	remaining: 303ms
5:	learn: 0.4366238	total: 33.6ms	remaining: 302ms
6:	learn: 0.4178187	total: 37ms	remaining: 280ms
7:	learn: 0.3995114	total: 43.3ms	remaining: 281ms
8:	learn: 0.3843629	total: 49.5ms	remaining: 281ms
9:	learn: 0.3696717	total: 55.5ms	remaining: 277ms
10:	learn: 0.3582542	total: 61.8ms	remaining: 275ms
11:	learn: 0.3466949	total: 67.6ms	remaining: 271ms
12:	learn: 0.3358271	total: 74ms	remaining: 268ms
13:	learn: 0.3222578	total: 80.1ms	remaining: 263ms
14:	learn: 0.3129769	total: 86ms	remaining: 258ms
15:	learn: 0.3044460	total: 92.5ms	remaining: 254ms
16:	learn: 0.2967215	total: 98.9ms	remaining: 250ms
17:	learn: 0.2900106	total: 105ms	remaining: 245ms
18:	learn: 0.2831286	total: 111ms	remaining: 240ms
19:	learn: 0.2743025	total: 11

[I 2022-08-13 11:02:08,941] Trial 14 finished with value: 0.8598639455782313 and parameters: {'learning_rate': 0.09472935936702269, 'depth': 7, 'iterations': 60, 'l2_leaf_reg': 5}. Best is trial 2 with value: 0.8673469387755102.


{'learning_rate': 0.07543460265326024, 'depth': 6, 'iterations': 51, 'l2_leaf_reg': 3} 0.8673469387755102


# Logistic Regression 

In [68]:
results = model_selection.cross_val_score(linear_model.LogisticRegression(),X,y,cv = model_selection.StratifiedKFold())
print(results)
print('Mean:',np.mean(results))


[0.84013605 0.84353741 0.84353741 0.84013605 0.84013605]
Mean: 0.8414965986394558


in Summary:

- the mean of accuracies for all algorithms was (between 83% and 86.5).
- at most all hyper parameter tuning techniques with the same algorithm not have big difference values for              accuracies.
- I have used Stratified Shuffle Split for cross validation score becuese targer values are inbalanced.
- I prefer to use aptuna ,hyperopt or same times gp_minimize more than gradesearch and randomizesdsearch                because the time complexty for the last tow is very very big but some times maybe i use one of theme they          give high accuracy compared to others.  
- for this dataset i will chose XGB as a Classifier with GridSearch as a hyper parameter tuning thats give 86%          accuracy.

# Final Model :

In [84]:
model = xgb.XGBClassifier(eta = 0.18000000000000005, sampling_method = 'uniform')
sss = model_selection.StratifiedShuffleSplit(n_splits = 15)
acc_list = []
for idx in sss.split(X = X, y = y):
    
    xtrain, ytrain = X[idx[0]], y[idx[0]] 
    xtest, ytest = X[idx[1]], y[idx[1]]
    
    model.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    print(metrics.confusion_matrix(ytest,y_pred))
    print(metrics.accuracy_score(ytest, y_pred))
    acc_list.append(metrics.accuracy_score(ytest, y_pred))
    print(metrics.classification_report(ytest,y_pred))
    print('--------------------------------------------------------')
print('Mean:',np.mean(acc_list))

[[117   6]
 [ 19   5]]
0.8299319727891157
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       123
           1       0.45      0.21      0.29        24

    accuracy                           0.83       147
   macro avg       0.66      0.58      0.59       147
weighted avg       0.79      0.83      0.80       147

--------------------------------------------------------
[[119   4]
 [ 18   6]]
0.8503401360544217
              precision    recall  f1-score   support

           0       0.87      0.97      0.92       123
           1       0.60      0.25      0.35        24

    accuracy                           0.85       147
   macro avg       0.73      0.61      0.63       147
weighted avg       0.82      0.85      0.82       147

--------------------------------------------------------
[[117   6]
 [ 12  12]]
0.8775510204081632
              precision    recall  f1-score   support

           0       0.91      0.95      0.93       1